In [ ]:
import requests

# Define the API endpoint and parameters
url = "https://upward-prime-turkey.ngrok-free.app/APIRest/plot/fetch_oma_api.php"
measurements_params = {
    "start_date": "2025-02-04",
    "end_date": "2025-02-04",
    "step_hours": 2
}

# Send the GET request to the API
response = requests.get(url, params=measurements_params)

# Check if the request was successful
if response.status_code == 200:
    measurements_data = response.json()  # Parse the JSON response
    print(f"Received {len(measurements_data)} records.")
else:
    print("Request failed:", response.status_code, response.text)

Received 5 records.


In [ ]:
measurements_data[0]

{'_id': {'$oid': '67a219c5d8c5de1bc124010a'},
 'OMA_id': 1,
 'timestamp_inicio': '2025-02-04 14:23:48',
 'timestamp_fin': '2025-02-04 14:38:48',
 'sensors': ['1', '2', '3', '4', '5', '6'],
 'modos': [{'modo_id': 1,
   'frequency': 3.8693880853627016,
   'phi': [{'real': 0.5376559203548802, 'imag': -0.016287253582152838},
    {'real': 0.08224067927253405, 'imag': -0.08875572780910136},
    {'real': 0.45269353734121215, 'imag': 0.007940078663948706},
    {'real': 0.9999999999999999, 'imag': 0},
    {'real': 0.13285234197771803, 'imag': -0.22874587614361797},
    {'real': 0.9727762283688149, 'imag': 0.00346152549984337}],
   'xi': 0.032889582965781865},
  {'modo_id': 3,
   'frequency': 5.8415799799549575,
   'phi': [{'real': 0.6586828853614725, 'imag': -0.06913727549113231},
    {'real': -0.5540601095000578, 'imag': -0.011493286934941348},
    {'real': -0.688572073696011, 'imag': 0.15048782778833283},
    {'real': 1, 'imag': 0},
    {'real': -1.0738326579553827, 'imag': 0.0857069166670160

In [50]:
url = "https://upward-prime-turkey.ngrok-free.app/APIRest/plot/fetch_th_api.php"
climate_params = {
    "limit": 50,
    "start_date": "2024-07-26",
    "end_date": "2024-12-04",
    "step_hours": 2
}

response = requests.get(url, params=climate_params)

if response.status_code == 200:
    climate_data = response.json()
    print(f"Received {len(climate_data)} environmental records.")
else:
    print("Request failed:", response.status_code, response.text)


Request failed: 429 {"error":"Has realizado demasiadas peticiones.","retry_after_minutes":10,"message":"Debes esperar 10 minuto(s) antes de volver a consultar."}


In [45]:
climate_data

{'error': 'No hay datos en ese intervalo de tiempo solicitado.',
 'available_data': {'from': '2024-07-26 11:00:00',
  'to': '2024-12-09 04:39:00'}}

# JSON to UNV

In [ ]:
measurements_data

[{'_id': {'$oid': '67a219c5d8c5de1bc124010a'},
  'OMA_id': 1,
  'timestamp_inicio': '2025-02-04 14:23:48',
  'timestamp_fin': '2025-02-04 14:38:48',
  'sensors': ['1', '2', '3', '4', '5', '6'],
  'modos': [{'modo_id': 1,
    'frequency': 3.8693880853627016,
    'phi': [{'real': 0.5376559203548802, 'imag': -0.016287253582152838},
     {'real': 0.08224067927253405, 'imag': -0.08875572780910136},
     {'real': 0.45269353734121215, 'imag': 0.007940078663948706},
     {'real': 0.9999999999999999, 'imag': 0},
     {'real': 0.13285234197771803, 'imag': -0.22874587614361797},
     {'real': 0.9727762283688149, 'imag': 0.00346152549984337}],
    'xi': 0.032889582965781865},
   {'modo_id': 3,
    'frequency': 5.8415799799549575,
    'phi': [{'real': 0.6586828853614725, 'imag': -0.06913727549113231},
     {'real': -0.5540601095000578, 'imag': -0.011493286934941348},
     {'real': -0.688572073696011, 'imag': 0.15048782778833283},
     {'real': 1, 'imag': 0},
     {'real': -1.0738326579553827, 'imag

In [30]:
import pyuff
import numpy as np

In [31]:
# Feltételezett szenzor koordináták (helyettesítsd a valós értékekkel, ha vannak)
sensor_coords = {
    '1': (4.7, 2.42, 23.52),
    '2': (21.206, 2.42, 23.52),
    '3': (18.206, 12.686, 23.52),
    '4': (10.0, 5.0, 23.52),
    '5': (15.0, 10.0, 23.52),
    '6': (20.0, 15.0, 23.52)
}

In [ ]:
import pyuff
import numpy as np

unv_data = []

# 1. UNV 164 - Units definition
units_data = pyuff.prepare_164(
    units_code=1,
    units_description='m/Newton',
    temp_mode=1,
    length=1.0,
    force=1.0,
    temp=1.0,
    temp_offset=1.0
)
unv_data.append(units_data)

# 2. UNV 15 - Node definitions
node_nums = []
x, y, z = [], [], []

for sensor_id in sorted(sensor_coords.keys(), key=int):
    node_nums.append(int(sensor_id))
    xi, yi, zi = sensor_coords[sensor_id]
    x.append(xi)
    y.append(yi)
    z.append(zi)

node_data = pyuff.prepare_15(
    node_nums=node_nums,
    def_cs=[0]*len(node_nums),
    disp_cs=[0]*len(node_nums),
    color=[0]*len(node_nums),
    x=x,
    y=y,
    z=z
)
unv_data.append(node_data)

# 3. UNV 55 - Mode shapes
for oma in measurements_data:
    sensors = oma['sensors']
    node_nums = np.array([int(s) for s in sensors])
    timestamp = oma['timestamp_inicio']

    modos = oma['modos']
    for idx, modo in enumerate(modos):
        freq = modo['frequency']
        modo_id = modo.get('modo_id', idx + 1)
        phi = modo['phi']
        
        r1 = np.array([complex(p['real'], p['imag']) for p in phi])
        r2 = np.zeros(len(r1))
        r3 = np.zeros(len(r1)) 
        
        mode_data = pyuff.prepare_55(
            id1=timestamp,
            id2='NONE',
            id3='NONE',
            id4='NONE',
            id5='NONE',
            model_type=1,
            analysis_type=2,
            data_ch=2,
            spec_data_type=8,
            data_type=2,
            n_data_per_node=3,
            load_case=1,
            mode_n=modo_id,
            freq=freq,
            modal_m=0.0,
            modal_damp_vis=0.0,
            modal_damp_his=0.0,
            node_nums=node_nums,
            r1=r1,
            r2=r2,
            r3=r3
        )
        unv_data.append(mode_data)

# UNV file saving
uff = pyuff.UFF('output.unv')
uff.write_sets(unv_data)


[{'real': 0.5376559203548802, 'imag': -0.016287253582152838}, {'real': 0.08224067927253405, 'imag': -0.08875572780910136}, {'real': 0.45269353734121215, 'imag': 0.007940078663948706}, {'real': 0.9999999999999999, 'imag': 0}, {'real': 0.13285234197771803, 'imag': -0.22874587614361797}, {'real': 0.9727762283688149, 'imag': 0.00346152549984337}]
[{'real': 0.6586828853614725, 'imag': -0.06913727549113231}, {'real': -0.5540601095000578, 'imag': -0.011493286934941348}, {'real': -0.688572073696011, 'imag': 0.15048782778833283}, {'real': 1, 'imag': 0}, {'real': -1.0738326579553827, 'imag': 0.08570691666701608}, {'real': -1.0876242471841788, 'imag': 0.048649437641591846}]
[{'real': 0.5204423176091175, 'imag': -0.0158953047951603}, {'real': 0.08506648562305738, 'imag': -0.06272334032356548}, {'real': 0.4639254151423202, 'imag': -0.03133445470767397}, {'real': 1, 'imag': 0}, {'real': 0.16231020072960978, 'imag': -0.1690118971664653}, {'real': 0.9679763943270129, 'imag': -0.02826810374315769}]
[{'

In [56]:
unv_data

[{'type': 164,
  'units_code': 1,
  'units_description': 'm/Newton',
  'temp_mode': 1,
  'length': 1.0,
  'force': 1.0,
  'temp': 1.0,
  'temp_offset': 1.0},
 {'type': 15,
  'node_nums': [1, 2, 3, 4, 5, 6],
  'def_cs': [0, 0, 0, 0, 0, 0],
  'disp_cs': [0, 0, 0, 0, 0, 0],
  'color': [0, 0, 0, 0, 0, 0],
  'x': [4.7, 21.206, 18.206, 10.0, 15.0, 20.0],
  'y': [2.42, 2.42, 12.686, 5.0, 10.0, 15.0],
  'z': [23.52, 23.52, 23.52, 23.52, 23.52, 23.52]},
 {'type': 55,
  'id1': '2025-02-04 14:23:48',
  'id2': 'NONE',
  'id3': 'NONE',
  'id4': 'NONE',
  'id5': 'NONE',
  'model_type': 1,
  'analysis_type': 2,
  'data_ch': 2,
  'spec_data_type': 8,
  'data_type': 2,
  'n_data_per_node': 3,
  'r1': array([0.53765592, 0.08224068, 0.45269354, 1.        , 0.13285234,
         0.97277623]),
  'r2': array([-0.01628725, -0.08875573,  0.00794008,  0.        , -0.22874588,
          0.00346153]),
  'r3': array([0., 0., 0., 0., 0., 0.]),
  'load_case': 1,
  'mode_n': 1,
  'freq': 3.8693880853627016,
  'modal_

In [63]:
def json2unv(json, sensor_coords, file_name,
                    units_code=1, units_description='mm/newton', temp_mode=1, length=1.0, force=1.0, temp=1.0, temp_offset=1.0,
                    def_cs=0, disp_cs=0, color=0,
                    id1='NONE', id2='NONE', id3='NONE', id4='NONE', id5='NONE', model_type=1, analysis_type=2, data_ch=2, 
                    spec_data_type=8, data_type=2, n_data_per_node=3, load_case=1, modal_m=0, modal_damp_vis=0., modal_damp_his=0.):
    unv_data = []

    ## Prepare type 164 data
    units_data = pyuff.prepare_164(
        units_code=units_code,
        units_description=units_description,
        temp_mode=temp_mode,
        length=length,
        force=force,
        temp=temp,
        temp_offset=temp_offset
    )
    unv_data.append(units_data)

    ## Prepare type 15 data
    node_nums = []
    x, y, z = [], [], []

    for sensor_id in sorted(sensor_coords.keys(), key=int):
        node_nums.append(int(sensor_id))
        xi, yi, zi = sensor_coords[sensor_id]
        x.append(xi)
        y.append(yi)
        z.append(zi)

    if def_cs==0:
        def_cs=[0]*len(node_nums)
    if disp_cs==0:
        disp_cs=[0]*len(node_nums)
    if color==0:
        color=[0]*len(node_nums)
    node_data = pyuff.prepare_15(
        node_nums=node_nums,
        def_cs=def_cs,
        disp_cs=disp_cs,
        color=color,
        x=x,
        y=y,
        z=z
    )
    unv_data.append(node_data)

    ## Prepare type 55 data
    for oma in json:
        sensors = oma['sensors']
        node_nums = np.array([int(s) for s in sensors])
        timestamp = oma['timestamp_inicio']
        if id1 == None:
            id1 = timestamp

        modos = oma['modos']
        for idx, modo in enumerate(modos):
            freq = modo['frequency']
            modo_id = modo.get('modo_id', idx + 1)
            phi = modo['phi']

            # Take the real and imaginary parts separately
            r1 = np.array([p['real'] for p in phi])
            r2 = np.array([p['imag'] for p in phi])
            r3 = np.zeros(len(r1))  # third component set to zero

            mode_data = pyuff.prepare_55(
                id1=id1,
                id2=id2,
                id3=id3,
                id4=id4,
                id5=id5,
                model_type=model_type,
                analysis_type=analysis_type,
                data_ch=data_ch,
                spec_data_type=spec_data_type,
                data_type=data_type,
                n_data_per_node=n_data_per_node,
                load_case=load_case,
                mode_n=modo_id,
                freq=freq,
                modal_m=modal_m,
                modal_damp_vis=modal_damp_vis,
                modal_damp_his=modal_damp_his,
                node_nums=node_nums,
                r1=r1,
                r2=r2,
                r3=r3
            )
            unv_data.append(mode_data)

    # UNV file saving
    print(unv_data)
    uff = pyuff.UFF(file_name)
    uff.write_sets(unv_data)


In [64]:
json2unv(measurements_data, sensor_coords, 'output.unv')

[{'type': 164, 'units_code': 1, 'units_description': 'mm/newton', 'temp_mode': 1, 'length': 1.0, 'force': 1.0, 'temp': 1.0, 'temp_offset': 1.0}, {'type': 15, 'node_nums': [1, 2, 3, 4, 5, 6], 'def_cs': [0, 0, 0, 0, 0, 0], 'disp_cs': [0, 0, 0, 0, 0, 0], 'color': [0, 0, 0, 0, 0, 0], 'x': [4.7, 21.206, 18.206, 10.0, 15.0, 20.0], 'y': [2.42, 2.42, 12.686, 5.0, 10.0, 15.0], 'z': [23.52, 23.52, 23.52, 23.52, 23.52, 23.52]}, {'type': 55, 'id1': 'NONE', 'id2': 'NONE', 'id3': 'NONE', 'id4': 'NONE', 'id5': 'NONE', 'model_type': 1, 'analysis_type': 2, 'data_ch': 2, 'spec_data_type': 8, 'data_type': 2, 'n_data_per_node': 3, 'r1': array([0.53765592, 0.08224068, 0.45269354, 1.        , 0.13285234,
       0.97277623]), 'r2': array([-0.01628725, -0.08875573,  0.00794008,  0.        , -0.22874588,
        0.00346153]), 'r3': array([0., 0., 0., 0., 0., 0.]), 'load_case': 1, 'mode_n': 1, 'freq': 3.8693880853627016, 'modal_m': 0, 'modal_damp_vis': 0.0, 'modal_damp_his': 0.0, 'node_nums': array([1, 2, 3, 4,